# Internet Search using Bing API - Bing Chat Clone

In this notebook, we'll delve into a sample for REACT framework for bing search agent in  **Bing Search Engine Agent**, utilizing both Langchain and the Azure Bing Search API service **harnessing agents and tools**, leveraging the capabilities of OpenAI's large language models (LLM), to perform the heavy lifting of reasoning and researching on our behalf.

In [1]:
import requests
from typing import Dict, List
from pydantic import BaseModel, Extra, root_validator

from langchain.chat_models import AzureChatOpenAI
from langchain.agents import AgentExecutor
from langchain.callbacks.manager import CallbackManager
from langchain.agents import initialize_agent, AgentType
from langchain.tools import BaseTool
from langchain.utilities import BingSearchAPIWrapper

from common.callbacks import StdOutCallbackHandler
from common.prompts import BING_PROMPT_PREFIX

from IPython.display import Markdown, HTML, display  

from dotenv import load_dotenv
load_dotenv("credentials.env")

def printmd(string):
    display(Markdown(string.replace("$","USD ")))

# GPT-4 models are necessary for this feature. GPT-35-turbo will make mistakes multiple times on following system prompt instructions.
MODEL_DEPLOYMENT_NAME = "gpt-35-turbo-16k" 

In [2]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_BASE"] = os.environ["OPENAI_ENDPOINT"]
os.environ["OPENAI_API_KEY"] = os.environ["OPENAI_API_KEY"]
os.environ["OPENAI_API_VERSION"] = os.environ["OPENAI_API_VERSION"]
os.environ["OPENAI_API_TYPE"] = "azure"

## Introduction to Callback Handlers

This following explanation comes directly from the Langchain documentation about Callbacks ([HERE](https://python.langchain.com/docs/modules/callbacks/)):

**Callbacks**:<br>
LangChain provides a callbacks system that allows you to hook into the various stages of your LLM application. This is useful for logging, monitoring, streaming, and other tasks. You can subscribe to these events by using the callbacks argument available throughout the API. This argument is list of handler objects.

**Callback handlers**:<br>
CallbackHandlers are objects that implement the CallbackHandler interface, which has a method for each event that can be subscribed to. The CallbackManager will call the appropriate method on each handler when the event is triggered.

--------------------
We will incorporate a handler for the callbacks, enabling us to observe the response as it streams and to gain insights into the Agent's reasoning process. This will prove incredibly valuable when we aim to stream the bot's responses to users and keep them informed about the ongoing process as they await the answer.

Our custom handler is on the folder `common/callbacks.py`. Go and take a look at it.

In [3]:
cb_handler = StdOutCallbackHandler()
cb_manager = CallbackManager(handlers=[cb_handler])

# Now we declare our LLM object with the callback handler 
llm = AzureChatOpenAI(deployment_name=MODEL_DEPLOYMENT_NAME, temperature=0, max_tokens=1000)

# or uncomment the below line if you want to see the responses being streamed
llm = AzureChatOpenAI(deployment_name=MODEL_DEPLOYMENT_NAME, temperature=0, max_tokens=1000, streaming=True, callback_manager=cb_manager)

## Creating a custom tool - Bing Search API tool

Langhain has already a pre-created tool called BingSearchAPIWrapper ([HERE](https://github.com/hwchase17/langchain/blob/master/langchain/utilities/bing_search.py)), however we are going to make it a bit better by using the results function instead of the run function, that way we not only have the text results, but also the title and link(source) of each snippet.

In [4]:
class MyBingSearch(BaseTool):
    """Tool for a Bing Search Wrapper"""
    
    name = "@bing"
    description = "useful when the questions includes the term: @bing.\n"

    k: int = 5
    
    def _run(self, query: str) -> str:
        bing = BingSearchAPIWrapper(k=self.k)
        return bing.results(query,num_results=self.k)
            
    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("This Tool does not support async")

Now, we create our REACT agent that uses our custom tool and our custom prompt `BING_PROMPT_PREFIX`

In [5]:
tools = [MyBingSearch(k=5)]
agent_executor = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         agent_kwargs={'prefix':BING_PROMPT_PREFIX}, callback_manager=cb_manager, )

Try some of the below questions, or others that you might like

In [ ]:
#QUESTION = "can I travel to Hawaii, Maui from Dallas, TX for 7 days with $7000 on the month of September, what are the best days to travel?"

In [8]:

QUESTION = "What is Emma Watson's age add 50?"

In [9]:
#As LLMs responses are never the same, we do a for loop in case the answer cannot be parsed according to our prompt instructions
for i in range(2):
    try:
        response = agent_executor.run(QUESTION) 
        break
    except Exception as e:
        response = str(e)
        continue

I need to find Emma Watson's age and add 50 to it.
Action: @bing
Action Input: "Emma Watson age"I need to find Emma Watson's age and add 50 to it.
Action: @bing
Action Input: "Emma Watson age"
I have found multiple sources that provide Emma Watson's age. According to these sources, Emma Watson was born on April 15, 1990. To calculate her age, we can subtract her birth year from the current year. Let's do the math: 

Current year: 2022
Emma Watson's birth year: 1990

2022 - 1990 = 32

Emma Watson is currently 32 years old. 

To add 50 to her age, we can simply add 50 to the result:

32 + 50 = 82

So, if we add 50 to Emma Watson's age, the result is 82.I need to find Emma Watson's age and add 50 to it.
Action: @bing
Action Input: "Emma Watson age"I need to find Emma Watson's age and add 50 to it.
Action: @bing
Action Input: "Emma Watson age"
I have found multiple sources that provide Emma Watson's age. According to these sources, Emma Watson was born on April 15, 1990. To calculate her a